In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import os 

# CHARGEMENT DES DONNEES 

Ajouter dans les données les grèves

In [2]:
dialect = 'mysql+pymysql'
user = 'root'
password = os.getenv('mdp_mySQL')
host = 'localhost'
port = 3306
database_idf = 'SNCF_TGV_TER'

engine_url = f"{dialect}://{user}:{password}@{host}:{port}/{database_idf}"
engine = create_engine(engine_url)

In [3]:
df_disruption = pd.read_sql('disruptions_tgv_ter',engine)
df_disruption

,id_disruption,vehicle_id,train_type,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,data_date
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,2024-03-18
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,2024-03-18
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,2024-03-18
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,TGV,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,2024-03-18
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,TGV,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,2024-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37984,506a4d2a-c51d-4af2-b40c-f4951e205015,SNCF:2024-03-24:9826:1187:LongDistanceTrain,TGV,stop_point:SNCF:87688887:LongDistanceTrain,Montpellier Sud de France,3.924010,43.5951,2024-03-24 10:17:00,2024-03-24 16:03:00,10.0,10.0,Régulation du trafic,2024-03-24
37985,506a4d2a-c51d-4af2-b40c-f4951e205015,SNCF:2024-03-24:9826:1187:LongDistanceTrain,TGV,stop_point:SNCF:87703975:LongDistanceTrain,Nîmes Pont-du-Gard,4.507680,43.8168,2024-03-24 10:17:00,2024-03-24 16:03:00,10.0,10.0,Régulation du trafic,2024-03-24
37986,506a4d2a-c51d-4af2-b40c-f4951e205015,SNCF:2024-03-24:9826:1187:LongDistanceTrain,TGV,stop_point:SNCF:87723197:LongDistanceTrain,Lyon Part Dieu,4.859410,45.7606,2024-03-24 10:17:00,2024-03-24 16:03:00,10.0,5.0,Régulation du trafic,2024-03-24
37987,506a4d2a-c51d-4af2-b40c-f4951e205015,SNCF:2024-03-24:9826:1187:LongDistanceTrain,TGV,stop_point:SNCF:87763029:LongDistanceTrain,Valence TGV Rhône-Alpes Sud,4.978650,44.9919,2024-03-24 10:17:00,2024-03-24 16:03:00,10.0,5.0,Régulation du trafic,2024-03-24


In [4]:
df_vehicle = pd.read_sql('vehicle_journeys_tgv_ter', engine)
df_vehicle['vehicle_id'] = df_vehicle['vehicle_id'].apply(lambda x: ':'.join(x.split(':')[1:]))
df_vehicle

,vehicle_id,route_id,time_begin,time_end,train_type,id_disruption,data_date
0,SNCF:2024-03-18:117752:1187:LongDistanceTrain,route:SNCF:FR:Line::B10C45A0-C32C-4232-85F2-4B...,05:00:00,07:19:00,Train grande vitesse,0f4e5f2c-a148-438f-b7cb-71b80199e8d4,2024-03-18
1,SNCF:2024-03-18:12244:1187:LongDistanceTrain,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,300d66cc-2614-4087-89d7-89abe997e2cf,2024-03-18
2,SNCF:2024-03-18:12251:1187:LongDistanceTrain,route:SNCF:CSR:421400,07:04:00,09:14:00,Train grande vitesse,None,2024-03-18
3,SNCF:2024-03-18:12254:1187:LongDistanceTrain,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,2024-03-18
4,SNCF:2024-03-18:12255:1187:LongDistanceTrain,route:SNCF:CSR:421400,11:04:00,13:14:00,Train grande vitesse,None,2024-03-18
...,...,...,...,...,...,...,...
55057,SNCF:2024-03-24:TC2324:1187:Train,route:SNCF:1187-87755009-87755371,11:00:00,11:22:00,TER / Intercités,None,2024-03-24
55058,SNCF:2024-03-24:TC2325:1187:Train,route:SNCF:1187-87755009-87755371,13:50:00,14:13:00,TER / Intercités,None,2024-03-24
55059,SNCF:2024-03-24:TC2326:1187:Train,route:SNCF:1187-87755009-87755371,15:20:00,15:42:00,TER / Intercités,None,2024-03-24
55060,SNCF:2024-03-24:TC2327:1187:Train,route:SNCF:1187-87755009-87755371,17:20:00,17:46:00,TER / Intercités,None,2024-03-24


In [5]:
df_lines = pd.read_sql_table('pt_lines_tgv_ter', engine)
df_lines

,route_id,route_name,train_type,network_name,opening_time,closing_time
0,route:SNCF:0018-80100537-87271007,Dortmund Hbf - Paris Nord,Train grande vitesse,Eurostar,05:18:00,12:21:00
1,route:SNCF:0018-80101840-87271007,Essen Hbf - Paris Nord,Train grande vitesse,Eurostar,07:35:00,22:14:00
2,route:SNCF:0018-80154583-87271007,Köln - Hauptbahnhof - Paris Nord,Train grande vitesse,Eurostar,05:21:00,23:14:00
3,route:SNCF:0018-84000588-87111849,Amsterdam Centraal - Marne-la-Vallée - Chessy,Train grande vitesse,Eurostar,06:47:00,11:05:00
4,route:SNCF:0018-84000588-87271007,Amsterdam Centraal - Paris Nord,Train grande vitesse,Eurostar,06:11:00,23:43:00
...,...,...,...,...,...,...
363,route:stop_area:SNCF:87113001_stop_area:SNCF:8...,Paris Est - Strasbourg,Train grande vitesse,additional service,None,None
364,route:stop_area:SNCF:87394007_stop_area:SNCF:8...,Chartres - Versailles Chantiers,Train grande vitesse,additional service,None,None
365,route:stop_area:SNCF:87411017_stop_area:SNCF:8...,Rouen Rive Droite - Paris Saint-Lazare,Train grande vitesse,additional service,None,None
366,route:stop_area:SNCF:87411017_stop_area:SNCF:8...,Rouen Rive Droite - Le Havre,Train grande vitesse,additional service,None,None


# PREPARATION DES DONNEES 

## Merge des tables 

Probleme que je ne comprend pas encore : il y a beaucoup plus de perturbation dans df_vehicle que dans df_disruption

In [6]:
df_merge_inter = pd.merge(df_disruption.drop(['data_date','train_type'],axis=1), df_vehicle, on=['vehicle_id','id_disruption'], how='left')
df_merge = pd.merge(df_merge_inter, df_lines.drop('train_type', axis=1), on='route_id', how='left')

In [7]:
df_merge_clean = df_merge.dropna(subset=['arrival_delay', 'departure_delay'])
df_merge_clean

,id_disruption,vehicle_id,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,route_id,time_begin,time_end,train_type,data_date,route_name,network_name,opening_time,closing_time
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37984,506a4d2a-c51d-4af2-b40c-f4951e205015,SNCF:2024-03-24:9826:1187:LongDistanceTrain,stop_point:SNCF:87688887:LongDistanceTrain,Montpellier Sud de France,3.924010,43.5951,2024-03-24 10:17:00,2024-03-24 16:03:00,10.0,10.0,Régulation du trafic,route:SNCF:CSR:001200,10:17:00,15:53:00,Train grande vitesse,2024-03-24,Marseille Saint-Charles - Bruxelles-Midi / Bru...,TGV INOUI,05:28:00,23:55:00
37985,506a4d2a-c51d-4af2-b40c-f4951e205015,SNCF:2024-03-24:9826:1187:LongDistanceTrain,stop_point:SNCF:87703975:LongDistanceTrain,Nîmes Pont-du-Gard,4.507680,43.8168,2024-03-24 10:17:00,2024-03-24 16:03:00,10.0,10.0,Régulation du trafic,route:SNCF:CSR:001200,10:17:00,15:53:00,Train grande vitesse,2024-03-24,Marseille Saint-Charles - Bruxelles-Midi / Bru...,TGV INOUI,05:28:00,23:55:00
37986,506a4d2a-c51d-4af2-b40c-f4951e205015,SNCF:2024-03-24:9826:1187:LongDistanceTrain,stop_point:SNCF:87723197:LongDistanceTrain,Lyon Part Dieu,4.859410,45.7606,2024-03-24 10:17:00,2024-03-24 16:03:00,10.0,5.0,Régulation du trafic,route:SNCF:CSR:001200,10:17:00,15:53:00,Train grande vitesse,2024-03-24,Marseille Saint-Charles - Bruxelles-Midi / Bru...,TGV INOUI,05:28:00,23:55:00
37987,506a4d2a-c51d-4af2-b40c-f4951e205015,SNCF:2024-03-24:9826:1187:LongDistanceTrain,stop_point:SNCF:87763029:LongDistanceTrain,Valence TGV Rhône-Alpes Sud,4.978650,44.9919,2024-03-24 10:17:00,2024-03-24 16:03:00,10.0,5.0,Régulation du trafic,route:SNCF:CSR:001200,10:17:00,15:53:00,Train grande vitesse,2024-03-24,Marseille Saint-Charles - Bruxelles-Midi / Bru...,TGV INOUI,05:28:00,23:55:00


### Rajout des trajets non perturbé

In [8]:
df_vehicle_add = df_vehicle[df_vehicle['id_disruption'].isna()]
df_vehicle_add

,vehicle_id,route_id,time_begin,time_end,train_type,id_disruption,data_date
2,SNCF:2024-03-18:12251:1187:LongDistanceTrain,route:SNCF:CSR:421400,07:04:00,09:14:00,Train grande vitesse,None,2024-03-18
4,SNCF:2024-03-18:12255:1187:LongDistanceTrain,route:SNCF:CSR:421400,11:04:00,13:14:00,Train grande vitesse,None,2024-03-18
6,SNCF:2024-03-18:12259:1187:LongDistanceTrain,route:SNCF:CSR:421400,12:04:00,14:14:00,Train grande vitesse,None,2024-03-18
7,SNCF:2024-03-18:12260:1187:LongDistanceTrain,route:SNCF:CSR:421400,14:46:00,17:03:00,Train grande vitesse,None,2024-03-18
9,SNCF:2024-03-18:12265:1187:LongDistanceTrain,route:SNCF:CSR:421400,13:58:00,16:14:00,Train grande vitesse,None,2024-03-18
...,...,...,...,...,...,...,...
55057,SNCF:2024-03-24:TC2324:1187:Train,route:SNCF:1187-87755009-87755371,11:00:00,11:22:00,TER / Intercités,None,2024-03-24
55058,SNCF:2024-03-24:TC2325:1187:Train,route:SNCF:1187-87755009-87755371,13:50:00,14:13:00,TER / Intercités,None,2024-03-24
55059,SNCF:2024-03-24:TC2326:1187:Train,route:SNCF:1187-87755009-87755371,15:20:00,15:42:00,TER / Intercités,None,2024-03-24
55060,SNCF:2024-03-24:TC2327:1187:Train,route:SNCF:1187-87755009-87755371,17:20:00,17:46:00,TER / Intercités,None,2024-03-24


In [9]:
#df_vehicle_add_merge  = pd.merge(df_vehicle_add,df_lines.drop('train_type', axis=1), on='route_id', how='left')
#df_vehicle_add_merge

In [10]:
df_vehicle_add.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47779 entries, 2 to 55061
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   vehicle_id     47779 non-null  object        
 1   route_id       47779 non-null  object        
 2   time_begin     47779 non-null  object        
 3   time_end       47779 non-null  object        
 4   train_type     47779 non-null  object        
 5   id_disruption  0 non-null      object        
 6   data_date      47779 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 2.9+ MB


In [11]:
df_final = pd.concat([df_merge_inter, df_vehicle_add], ignore_index=True)
df_final

,id_disruption,vehicle_id,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,route_id,time_begin,time_end,train_type,data_date
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85763,None,SNCF:2024-03-24:TC2324:1187:Train,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,11:00:00,11:22:00,TER / Intercités,2024-03-24
85764,None,SNCF:2024-03-24:TC2325:1187:Train,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,13:50:00,14:13:00,TER / Intercités,2024-03-24
85765,None,SNCF:2024-03-24:TC2326:1187:Train,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,15:20:00,15:42:00,TER / Intercités,2024-03-24
85766,None,SNCF:2024-03-24:TC2327:1187:Train,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,17:20:00,17:46:00,TER / Intercités,2024-03-24


## Choix des variables

In [12]:
df_final['disrupted'] = np.where(df_final['id_disruption'].isna(), 0, 1)

In [13]:
df_final['week_day'] = df_final['data_date'].dt.weekday

In [14]:
df_final_v2 = df_final[[#'id_disruption', 
        #'vehicle_id', 
        'id_stop', 
        #'name_stop', 
        #'lon', 
        #'lat',
        'disruption_start',
        'disruption_end', 
        'arrival_delay',
        'departure_delay', 
        'cause_delay',
        'route_id',
        'time_begin',
        'time_end',
        'train_type',
        #'data_date',
        'disrupted',
        'week_day']]

In [15]:
# Création de valeurs binaires pour chaque type de train et jour de la semaine
df_final_v2 = pd.get_dummies(df_final_v2, columns=['train_type','week_day'])

In [16]:
# On créer des tranches d'heures puis indique si le train a circulé dans cette tranche
tranches = list(range(4*60, 25*60, 60))

def train_dans_tranche(depart, arrivee, tranche_debut, tranche_fin):
    return not (arrivee <= tranche_debut or depart >= tranche_fin)

for i in range(len(tranches)-1):
    tranche_nom = f'{tranches[i]//60:02d}h-{tranches[i+1]//60:02d}h'
    df_final_v2[tranche_nom] = df_final_v2.apply(lambda x: train_dans_tranche(x['time_begin'].hour*60 + x['time_begin'].minute, x['time_end'].hour*60 + x['time_end'].minute, tranches[i], tranches[i+1]), axis=1)


In [17]:
# On remplace les trajet par leur fréquence respective
route_freq = df_final_v2['route_id'].value_counts() / len(df_final_v2)
df_final_v2['route_freq'] = df_final_v2['route_id'].map(route_freq)

In [18]:
df_final_v2 = df_final_v2.drop(['time_begin', 'time_end','route_id'], axis=1)

In [19]:
df_final_v2

,id_stop,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,disrupted,train_type_TER / Intercités,train_type_Train grande vitesse,week_day_0,...,15h-16h,16h-17h,17h-18h,18h-19h,19h-20h,20h-21h,21h-22h,22h-23h,23h-24h,route_freq
0,stop_point:SNCF:87391003:LongDistanceTrain,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,1,False,True,True,...,False,False,False,True,True,True,False,False,False,0.003673
1,stop_point:SNCF:87393702:LongDistanceTrain,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,1,False,True,True,...,False,False,False,True,True,True,False,False,False,0.003673
2,stop_point:SNCF:87581009:LongDistanceTrain,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,1,False,True,True,...,False,False,False,True,True,True,False,False,False,0.003673
3,stop_point:SNCF:87391003:LongDistanceTrain,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,1,False,True,True,...,False,False,False,False,False,False,False,False,False,0.003673
4,stop_point:SNCF:87581009:LongDistanceTrain,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,1,False,True,True,...,False,False,False,False,False,False,False,False,False,0.003673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85763,NaN,NaT,NaT,NaN,NaN,NaN,0,True,False,False,...,False,False,False,False,False,False,False,False,False,0.000117
85764,NaN,NaT,NaT,NaN,NaN,NaN,0,True,False,False,...,False,False,False,False,False,False,False,False,False,0.000117
85765,NaN,NaT,NaT,NaN,NaN,NaN,0,True,False,False,...,True,False,False,False,False,False,False,False,False,0.000117
85766,NaN,NaT,NaT,NaN,NaN,NaN,0,True,False,False,...,False,False,True,False,False,False,False,False,False,0.000117


# RANDOM FOREST (CLASSIFICATION)

In [20]:
df_randfo = df_final_v2.drop(['id_stop',
                'disruption_start',
                'disruption_end',
                'arrival_delay',
                'departure_delay',
                'cause_delay',
                'train_type_Train grande vitesse',
                'train_type_TER / Intercités'],
                             axis=1)

In [21]:
df_randfo

,disrupted,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4,week_day_5,week_day_6,04h-05h,05h-06h,...,15h-16h,16h-17h,17h-18h,18h-19h,19h-20h,20h-21h,21h-22h,22h-23h,23h-24h,route_freq
0,1,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,0.003673
1,1,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,0.003673
2,1,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,0.003673
3,1,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.003673
4,1,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.003673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85763,0,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,0.000117
85764,0,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,0.000117
85765,0,False,False,False,False,False,False,True,False,False,...,True,False,False,False,False,False,False,False,False,0.000117
85766,0,False,False,False,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,0.000117


In [22]:
X_randfo = df_randfo.drop('disrupted', axis=1)
y_randfo = df_randfo['disrupted']

X_train_randfo, X_test_randfo, y_train_randfo, y_test_randfo = train_test_split(X_randfo, y_randfo, test_size=0.2, random_state=42)

In [23]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_randfo, y_train_randfo)

RandomForestClassifier(random_state=42)

In [24]:
# Prédictions
predictions = rf.predict(X_test_randfo)

# Évaluation
print(classification_report(y_test_randfo, predictions))
print("ROC AUC score:", roc_auc_score(y_test_randfo, predictions))

              precision    recall  f1-score   support

           0       0.99      0.89      0.94      9540
           1       0.87      0.99      0.93      7614

    accuracy                           0.93     17154
   macro avg       0.93      0.94      0.93     17154
weighted avg       0.94      0.93      0.93     17154

ROC AUC score: 0.93781626199999


###  Ajustements des paramètres

In [30]:
param_grid = {
    'n_estimators': [100, 200,300],  
    'max_depth': [None, 10, 20,30],  
}

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train_randfo, y_train_randfo)

print("Meilleurs paramètres:", grid_search.best_params_)

Meilleurs paramètres: {'max_depth': 30, 'n_estimators': 300}


In [31]:
rf_best = RandomForestClassifier(**grid_search.best_params_,random_state=42)
rf_best.fit(X_train_randfo, y_train_randfo)

RandomForestClassifier(max_depth=30, n_estimators=300, random_state=42)

In [32]:
# Prédictions
predictions_best = rf_best.predict(X_test_randfo)

# Évaluation
print(classification_report(y_test_randfo, predictions_best))
print("ROC AUC score:", roc_auc_score(y_test_randfo, predictions_best))

              precision    recall  f1-score   support

           0       0.99      0.89      0.94      9540
           1       0.87      0.99      0.93      7614

    accuracy                           0.93     17154
   macro avg       0.93      0.94      0.93     17154
weighted avg       0.94      0.93      0.93     17154

ROC AUC score: 0.9375270727706163


###  Importance des diffrétentes variables

In [33]:
importances = rf_best.feature_importances_

features = X_train_randfo.columns
importance_scores = sorted(list(zip(features, importances)), key=lambda x: x[1], reverse=True)

for feature, importance in importance_scores:
    print(f"{feature}: {importance}")

route_freq: 0.770951170431935
19h-20h: 0.012445593838488245
20h-21h: 0.01177119444001039
18h-19h: 0.01156866469506189
week_day_4: 0.011429052350776334
week_day_0: 0.011092359586536761
17h-18h: 0.009947855352301409
week_day_2: 0.009679100158522266
week_day_6: 0.009530937016818236
week_day_1: 0.009482734195108445
week_day_5: 0.009445849599079624
week_day_3: 0.009443027235978103
15h-16h: 0.009224753922117988
21h-22h: 0.008973135271036997
16h-17h: 0.008860506264373716
08h-09h: 0.008591777451453607
13h-14h: 0.008306985132895366
14h-15h: 0.00817946379934924
10h-11h: 0.008166686511555539
09h-10h: 0.008137822119349183
12h-13h: 0.008065022425772611
07h-08h: 0.00729508115319851
11h-12h: 0.0071477832039231245
06h-07h: 0.006574325905239932
22h-23h: 0.005881533305955324
05h-06h: 0.005782349133340193
23h-24h: 0.0029459032160272488
04h-05h: 0.0010793322837946429


séparer semaine et weekend 